In [2]:
import pandas as pd
import  numpy as np

### Загрузка данных

In [3]:
df = pd.read_csv("ranking_dataset.csv")
df.head()

,id,authour_name,title,address,issn,journal,year
0,WOS:000209908400008,"Tamosiunaite, Aurelija",Defining 'Lithuanian' Orthographic debates at ...,"Vytautas Magnus Univ, Fac Humanities, Dept Lit...",1387-6732,WRITTEN LANGUAGE AND LITERACY,2015
1,WOS:000209919200006,"Zhang, Hui Andy",RECOGNITION THROUGH REINVENTION: THE MYTH OF C...,"Northwestern Univ, Evanston, IL 60208 USA.",2013-3294,452 F-REVISTA DE TEORIA DE LA LITERATURA Y LIT...,2015
2,WOS:000210161400001,"Bortnik, Boris",Interdisciplinary project-based learning: tech...,"Ural State Univ Econ, Ekaterinburg, Russia.",2156-7069,RESEARCH IN LEARNING TECHNOLOGY,2015
3,WOS:000210161400001,"Mironova, Ludmila",Interdisciplinary project-based learning: tech...,"Ural State Univ Econ, Ekaterinburg, Russia.",2156-7069,RESEARCH IN LEARNING TECHNOLOGY,2015
4,WOS:000210161400001,"Podshivalova, Ekaterina",Interdisciplinary project-based learning: tech...,"Ural State Univ Econ, Ekaterinburg, Russia.",2156-7069,RESEARCH IN LEARNING TECHNOLOGY,2015


In [4]:
df.shape

(21788, 7)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21788 entries, 0 to 21787
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            21788 non-null  object
 1   authour_name  21788 non-null  object
 2   title         21788 non-null  object
 3   address       21788 non-null  object
 4   issn          21302 non-null  object
 5   journal       21788 non-null  object
 6   year          21788 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 1.2+ MB


### Проверка данных на дубликаты, пропущенные/неверные значения

In [6]:
df.isnull().sum() # missing/NaN values for each column

id                0
authour_name      0
title             0
address           0
issn            486
journal           0
year              0
dtype: int64

В issn имеюся пропущенные значения, но для необхдимых расчетов он не используется

### Добавление столбцов org_name и  country выделением из стобца address

In [7]:
org_name_list = []
country_list = []
for address in df['address'].values:
    temp_list = address.split(', ')
    org_name_list.append(temp_list[0]) # appends organization name
    country_list.append(temp_list[-1][:-1]) # [:-1] removes the period '.' at the end of country name

In [8]:
df['org_name'] = org_name_list
df['country'] = country_list

In [9]:
df.head()

,id,authour_name,title,address,issn,journal,year,org_name,country
0,WOS:000209908400008,"Tamosiunaite, Aurelija",Defining 'Lithuanian' Orthographic debates at ...,"Vytautas Magnus Univ, Fac Humanities, Dept Lit...",1387-6732,WRITTEN LANGUAGE AND LITERACY,2015,Vytautas Magnus Univ,Lithuania
1,WOS:000209919200006,"Zhang, Hui Andy",RECOGNITION THROUGH REINVENTION: THE MYTH OF C...,"Northwestern Univ, Evanston, IL 60208 USA.",2013-3294,452 F-REVISTA DE TEORIA DE LA LITERATURA Y LIT...,2015,Northwestern Univ,IL 60208 USA
2,WOS:000210161400001,"Bortnik, Boris",Interdisciplinary project-based learning: tech...,"Ural State Univ Econ, Ekaterinburg, Russia.",2156-7069,RESEARCH IN LEARNING TECHNOLOGY,2015,Ural State Univ Econ,Russia
3,WOS:000210161400001,"Mironova, Ludmila",Interdisciplinary project-based learning: tech...,"Ural State Univ Econ, Ekaterinburg, Russia.",2156-7069,RESEARCH IN LEARNING TECHNOLOGY,2015,Ural State Univ Econ,Russia
4,WOS:000210161400001,"Podshivalova, Ekaterina",Interdisciplinary project-based learning: tech...,"Ural State Univ Econ, Ekaterinburg, Russia.",2156-7069,RESEARCH IN LEARNING TECHNOLOGY,2015,Ural State Univ Econ,Russia


### Количество публикаций в названии которых встречается Russia или Russian

In [10]:
boolean_pandas_series  = df['title'].drop_duplicates().str.lower().str.contains('russia|russian')
boolean_pandas_series.values.sum()

4859

Функция drop_duplicates() убирает публикации с одним и тем же названием,
lower() делает все буквы строчными (позволяя вместо случаев Russia, Russian, russian, RUSSIA, RUSSIAN 
рассматривать только russia и russian)

### Cписок ТОП-20 организаций по количеству авторов с указанием их количества

In [11]:
df1 = df[['authour_name', 'org_name']]
grouped = df1.groupby('org_name')
result_table_1 = grouped.aggregate({'authour_name': "nunique"}).reset_index() # takes into account only different authors
result_table_1.rename(columns = {'authour_name' : 'number_of_authours'}, inplace = True)
result_table_1.sort_values(by=['number_of_authours'], ascending = False, inplace= True)
result_table_1.head(20).to_excel("Top_20_organizations_by_number_of_authors.xlsx", index = False) 

### Cписок ТОП-20 авторов по количеству публикаций с указанием их количества

In [12]:
df2 = df[['id', 'authour_name']]
result_table_2 = df2.groupby(['authour_name']).size().reset_index(name='number_of_publications')
result_table_2.sort_values(by=['number_of_publications'], ascending = False, inplace= True)
result_table_2.head(20).to_excel("Top_20_authors_by_number_of_publications.xlsx", index = False) 

### Cписок ТОП-20 организаций по количеству публикаций с указанием их количества

In [13]:
df3 = df[['id', 'org_name']]
result_table_3 = df3.groupby(['org_name']).size().reset_index(name='number_of_publications')
result_table_3.sort_values(by=['number_of_publications'], ascending = False, inplace= True)
result_table_3.head(20).to_excel("Top_20_organizations_by_number_of_publications.xlsx", index = False) 

### Cписок ТОП-20 журналов по количеству публикаций с указанием их количества

In [14]:
df4 = df[['id', 'journal']]
result_table_4 = df4.groupby(['journal']).size().reset_index(name='number_of_publications')
result_table_4.sort_values(by=['number_of_publications'], ascending = False, inplace= True)
result_table_4.head(20).to_excel("Top_20_journals_by_number_of_publications.xlsx", index = False) 